# Packager la solution dans une interface Streamlit

Voilà. Vous avez créé avec succès un assistant de recherche utilisant des données d'articles scientifiques à l'aide de Gemini, LangChain et Chroma. Il est maintenant temps de packager cette solution dans une interface Streamlit.

**Avertissement :** Cette interface Streamlit est présentée et exécutée ici au sein d'un notebook Google Colab à titre exceptionnel pour en simplifier l'accès et l'utilisation, notamment pour les personnes ne disposant pas d'un environnement Python ou d'un IDE configuré sur leur machine locale. En règle générale, une application Streamlit est conçue pour être exécutée en local sur votre ordinateur ou déployée sur un serveur web dédié pour un accès public. Vous pouvez exécuter cette même interface en local en installant les dépendances nécessaires et en utilisant la commande `streamlit run app.py`.

## Installation des packages nécessaires

In [4]:
!pip install -U -q langchain-community
!pip install -q langchain-google-genai
!pip install -q chromadb
!pip install -q pypdf
!pip install -q streamlit
!npm install -q localtunnel

'npm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Si vous utlisez ce notebook en local, lancer l'interface streamlit avec la commande dédiée et ne pas avec le package `localtunnel`.

## Création de l'interface avec Streamlit

In [1]:
%%writefile app.py
import streamlit as st
import time # Optional: To simulate bot thinking time
from rag import get_llm_response, generate_vectorstore
import os
# --- Configuration ---
st.set_page_config(page_title="My Basic Chatbot", layout="centered")
st.title("🤖 Simple Chatbot Interface")
uploaded_pdf_file = st.sidebar.file_uploader("Upload the pdf file", type="pdf", label_visibility="collapsed")

# if uploaded_file is None and st.session_state.count ==0:
#     st.write("Please upload the file to see the final results.\n")
if 'count' not in st.session_state:
	st.session_state.count = 0

if uploaded_pdf_file and st.session_state.count < 1:
    with st.spinner("Processing..."):
        DIR_NAME = "input_data"
        if not os.path.exists(DIR_NAME):
            os.makedirs(DIR_NAME)

        pdf_file_path = os.path.join(DIR_NAME, uploaded_pdf_file.name)

        with open(pdf_file_path, "wb") as f:
            f.write(uploaded_pdf_file.getbuffer())

        generate_vectorstore(pdf_file_path)
# --- Placeholder for Chatbot Logic ---
# Replace this with your actual chatbot function/API call
def get_bot_response(user_message):
    """
    Simulates a bot response.
    Replace this with your actual chatbot logic (e.g., API call, model inference).
    """
    # Simulate thinking time (optional)
    # time.sleep(0.5)

    # Basic echo response for demonstration
    # return f"You said: '{user_message}'"

    # Slightly more interactive placeholder
    if "hello" in user_message.lower():
        return "Hello there! How can I help you today?"
    elif "how are you" in user_message.lower():
        return "I'm just a bunch of code, but I'm running smoothly! Thanks for asking."
    elif "bye" in user_message.lower():
        return "Goodbye! Have a great day."
    else:
        return get_llm_response(user_message)

# --- Session State Initialization ---
# Initialize chat history if it doesn't exist
if "messages" not in st.session_state:
    st.session_state.messages = []
    # Optional: Add a starting welcome message from the assistant
    st.session_state.messages.append(
        {"role": "assistant", "content": "Hi! Ask me anything."}
    )


# --- Display Chat History ---
st.write("--- Conversation History ---")
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"]) # Use markdown for potential formatting
st.write("---") # Separator


# --- Handle User Input ---
# Use st.chat_input which is designed specifically for chat interfaces
prompt = st.chat_input("What is up?")

if prompt:
    # 1. Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # 2. Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # 3. Get bot response (using placeholder function)
    with st.spinner("Thinking..."): # Show a thinking indicator
        bot_response = get_bot_response(prompt)

    # 4. Add bot response to chat history
    st.session_state.messages.append({"role": "assistant", "content": bot_response})

    # 5. Display assistant response in chat message container
    # Use st.rerun() for a smoother update after adding messages
    st.rerun()


# --- Optional: Add a button to clear history ---
if st.button("Clear Chat History"):
    st.session_state.messages = [{"role": "assistant", "content": "Chat cleared. How can I help?"}]
    st.rerun()

Writing app.py


## Implémentation de la solution

In [ ]:
%%writefile rag.py
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
import getpass
from langchain import PromptTemplate
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

os.environ['GOOGLE_API_KEY'] = "XXXXXXXXXXXX" # Ici remplacez par votre API Key !!!
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.7, top_p=0.85)

def get_retriver(gemini_embeddings):

    vectorstore_disk = Chroma(
                         persist_directory="./chroma_db",
                         embedding_function=gemini_embeddings
                       )
    return vectorstore_disk.as_retriever(search_kwargs={"k": 3}) # k=3 signifie récupérer les 5 chunks les plus pertinents

def get_prompt():

    llm_prompt_template = """You are an assistant for question-answering tasks.
    Use the following context to answer the question.
    If you don't know the answer, just say that you don't know.
    Use five sentences maximum and keep the answer concise.\n
    Question: {question} \nContext: {context} \nAnswer:"""

    return PromptTemplate.from_template(llm_prompt_template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def get_llm_response(question):
    retriever = get_retriver(gemini_embeddings)
    llm_prompt = get_prompt()
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | llm_prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain.invoke(question)

def load_file(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages

def generate_vectorstore(file_path):

    pages = load_file(file_path)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunked_documents = text_splitter.split_documents(pages)

    print(f"Chargé {len(pages)} pages et créé {len(chunked_documents)} chunks.")
    vectorstore = Chroma.from_documents(
                        documents=chunked_documents,
                        embedding=gemini_embeddings,
                        persist_directory="./chroma_db"
                    )

    print(f"Base de données vectorielle créée/mise à jour dans ./chroma_db avec {len(chunked_documents)} chunks.")


Writing rag.py


## Lancement de la solution

In [3]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

& �tait inattendu.
